In [1]:
# -*- coding: utf-8 -*-

import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace import sarimax
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pandas import Series
from statsmodels.tsa.arima_model import ARIMA 
os.chdir("C:/Users/arora/Documents/spark-python")  
os.environ['SPARK_HOME'] = 'C:/Users/arora/spark-2.3.3-bin-hadoop2.7'

# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. 
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.7-src.zip"))

from pyspark.sql import Row
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import split
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.functions import lit
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import TimestampType,IntegerType
from pyspark.sql.functions import lag
from datetime import datetime

def time_delta(y,x): 
    from datetime import datetime
    end = datetime.strptime(y, '%Y-%m-%dT%H:%M:%S.%fZ')  
    start = datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')
    delta = (end-start).total_seconds()
    return delta

In [2]:
#Create a Spark Session
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Nidhi") \
    .getOrCreate()
    
#Get the Spark Context from Spark Session    
sc = SpSession.sparkContext


In [4]:
func =  udf (lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ'), TimestampType())

func2 =  udf (lambda x: x.replace(microsecond=0), TimestampType())


split_col = split(logDataFrame['client_ip'], ':')
logDataFrame = logDataFrame.withColumn('IP', split_col.getItem(0))
logDataFrame.createOrReplaceTempView("data_With_IP")
logDataFrameWithIP = logDataFrame.withColumn('time_stamp', func(col('timestamp_str'))).cache()

logDataFrameWithIPNoMicroSecs = logDataFrameWithIP.withColumn('time_stamp_without_microsecs', func2(col('time_stamp'))).cache()
logDataFrameWithIPNoMicroSecs.take(1)
logDataFrameWithIPNoMicroSecs.createOrReplaceTempView("df_time_stamp_without_microsecs")

logDataFrameIPTimeStamp = logDataFrameWithIP.withColumn('time_stamp_previous',
                        lag(logDataFrameWithIP.time_stamp,1)
                                 .over(Window.partitionBy("IP").orderBy("time_stamp"))).cache()

In [6]:
logDataFrameIPTimeStamp = logDataFrameIPTimeStamp.withColumn(
    "time_diff_in_secs", 
    unix_timestamp("time_stamp") - unix_timestamp("time_stamp_previous")
)

logDataFrameIPTimeStamp.createOrReplaceTempView("log_session")

In [8]:
SpSession.sql("select IP,time_diff_in_secs from log_session where time_diff_in_secs>900 order by IP, time_stamp ").show()

+-------------+-----------------+
|           IP|time_diff_in_secs|
+-------------+-----------------+
|1.186.111.224|              980|
|1.186.180.183|            17305|
| 1.186.247.60|            15353|
| 1.186.247.60|            33322|
| 1.186.248.29|             4688|
| 1.186.28.220|            20022|
|  1.186.32.38|            40021|
|  1.186.32.38|             6131|
|   1.186.37.1|             6449|
|  1.186.37.11|            25461|
|  1.186.37.17|             5766|
|   1.186.37.2|            39363|
|   1.186.37.2|             6460|
|  1.186.37.25|             5267|
|  1.186.37.25|            20108|
|  1.186.37.25|             1599|
|   1.186.37.5|             1138|
|   1.186.38.1|            20609|
|  1.186.38.19|            33119|
|  1.186.38.25|            26582|
+-------------+-----------------+
only showing top 20 rows



In [13]:
logDataFrameSession = logDataFrameIPTimeStamp.select(F.when(logDataFrameIPTimeStamp.time_diff_in_secs > 900, lit(1)).otherwise(lit(0)).alias("new_session"))


logDataFrameSession = logDataFrameIPTimeStamp.withColumn('new_session',F.when(((logDataFrameIPTimeStamp.time_diff_in_secs > 900) | (logDataFrameIPTimeStamp.time_diff_in_secs.isNull())), lit(1)).otherwise(lit(0)))
logDataFrameSession.createOrReplaceTempView("log_new_session")


In [18]:
logDataFrameSessionId = SpSession.sql("select *,(SUM(new_session) OVER (PARTITION BY IP ORDER BY IP,time_stamp)) as session_id from log_new_session")
logDataFrameSessionId.createOrReplaceTempView("log_new_session_id")
SpSession.sql("select IP,new_session,session_id,time_stamp,ifnull(time_diff_in_secs, 0) as time_diff_in_secs ,time_stamp_previous from log_new_session_id order by IP,time_stamp").show(5)

#Q1 i.e the sessions are represented by logDataFrameSessionId, Every IP has its own session ids

+------------+-----------+----------+--------------------+-----------------+--------------------+
|          IP|new_session|session_id|          time_stamp|time_diff_in_secs| time_stamp_previous|
+------------+-----------+----------+--------------------+-----------------+--------------------+
|1.186.101.79|          1|         1|2015-07-22 10:45:...|                0|                null|
|1.186.101.79|          0|         1|2015-07-22 10:45:...|                0|2015-07-22 10:45:...|
|1.186.101.79|          0|         1|2015-07-22 10:46:...|               32|2015-07-22 10:45:...|
|1.186.101.79|          0|         1|2015-07-22 10:46:...|               29|2015-07-22 10:46:...|
|1.186.101.79|          0|         1|2015-07-22 10:47:...|                5|2015-07-22 10:46:...|
+------------+-----------+----------+--------------------+-----------------+--------------------+
only showing top 5 rows



In [20]:
SpSession.sql("select * from log_new_session_id where IP is null").count()
#So, null IPs are there

0

In [24]:
logDataFrameSessionTime = SpSession.sql("select IP,session_id,sum(time_diff_in_secs) as session_time from log_new_session_id group by IP,session_id order by IP,session_id")
logDataFrameSessionTime.createOrReplaceTempView("log_session_time_tmp")
logDataFrameSessionTime= SpSession.sql("select IP, ifnull(session_time, 0) as session_time from log_session_time_tmp")
logDataFrameSessionTime.show(5)

+-------------+------------+
|           IP|session_time|
+-------------+------------+
| 1.186.101.79|        1118|
|1.186.103.240|          11|
| 1.186.103.78|           9|
|1.186.108.213|         113|
|1.186.108.230|          19|
+-------------+------------+
only showing top 5 rows



In [25]:
###################################Q2###############################################
logDataFrameSessionTime.createOrReplaceTempView("log_session_time")
SpSession.sql("select Avg(session_time) as avg_session_time from log_session_time").show()
#average session time is 2175.9683673929717

+------------------+
|  avg_session_time|
+------------------+
|2175.9683673929717|
+------------------+



In [26]:
###################################Q3###############################################
SpSession.sql("select distinct IP,session_id, request from log_new_session_id").take(1)
logUniqueVisits = SpSession.sql("select IP,session_id,count(*) as unique_url_count from ( select distinct IP,session_id, url from log_new_session_id )group by IP,session_id order by IP,session_id")
logUniqueVisits.show()

+-------------+----------+----------------+
|           IP|session_id|unique_url_count|
+-------------+----------+----------------+
| 1.186.101.79|         1|              12|
|1.186.103.240|         1|               5|
| 1.186.103.78|         1|               4|
|1.186.108.213|         1|               2|
|1.186.108.230|         1|               5|
|1.186.108.242|         1|               3|
| 1.186.108.28|         1|              80|
| 1.186.108.29|         1|               3|
| 1.186.108.79|         1|               3|
|1.186.111.224|         1|               9|
|1.186.111.224|         2|              12|
| 1.186.112.12|         1|               3|
| 1.186.112.52|         1|              16|
|1.186.114.135|         1|               2|
| 1.186.114.45|         1|               6|
|1.186.117.150|         1|               1|
| 1.186.128.57|         1|               8|
|1.186.129.213|         1|               6|
|1.186.135.123|         1|              11|
|1.186.135.254|         1|      

In [ ]:
###################################Q4###############################################
SpSession.sql("select IP,session_time from log_session_time order by session_time desc").show()